In [20]:
# importing the libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, confusion_matrix

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stemmer = nltk.SnowballStemmer('english')
import string
import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_csv('/content/twitter.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


SO, after having a brief look at the data, we found that our tweets are classified into 3 different classes.

class=0; Negative Speech
class=1; Hateful Comment
class=2; Neutral Comment Therefore, now we will create one new column for our better understanding.

In [22]:
# we will create one new column ny the name of lable for classifying the tweets.
df['label'] = df['class'].map({0:'Negative Speech', 1:'Hateful Comment', 2:'Neutral Comment'})
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,label
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,Neutral Comment
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Hateful Comment
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Hateful Comment
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Hateful Comment
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Hateful Comment


In [23]:
# we will now all remove unnesscary columns from our data
df_new = df[['tweet', 'label']]
df_new.head()

,tweet,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,Neutral Comment
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Hateful Comment
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Hateful Comment
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Hateful Comment
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Hateful Comment


In [24]:
# setting the stopwords to  english
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# now we will create one function that removes all the punctuation marks and rest of special characters from the tweets.
def clean_text(text):
  text = str(text).lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  text = [w for w in text.split(' ') if w not in stopwords]
  text = " ".join(text)
  text = [stemmer.stem(w) for w in text.split(' ')]
  text = " ".join(text)
  return text

df_new['tweet'] = df_new['tweet'].apply(clean_text)

df_new.head()

,tweet,label
0,rt mayasolov woman shouldnt complain clean ho...,Neutral Comment
1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,Hateful Comment
2,rt urkindofbrand dawg rt ever fuck bitch sta...,Hateful Comment
3,rt cganderson vivabas look like tranni,Hateful Comment
4,rt shenikarobert shit hear might true might f...,Hateful Comment


In [26]:
# now we will see the amount of each labels in our dataset
df_new['label'].value_counts()

label
Hateful Comment    19190
Neutral Comment     4163
Negative Speech     1430
Name: count, dtype: int64

In [27]:
x = np.array(df_new['tweet'])
y = np.array(df_new['label'])

In [28]:
cv = CountVectorizer()
X = cv.fit_transform(x)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [30]:
X_train = X_train.toarray()
X_test = X_test.toarray()

In [31]:
model = DecisionTreeClassifier()



In [32]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [33]:
y_pred = model.predict(X_test)

In [34]:
accuracy_score(y_test, y_pred)

0.8711333903900232

In [39]:
i = "You are good human"
i = cv.transform([i]).toarray()
model.predict(i)

array(['Neutral Comment'], dtype=object)

In [42]:
text = input("Enter your text: ")
data = cv.transform([text]).toarray()
output=model.predict(data)
print(output)

Enter your text: You are one son of a bitch
['Hateful Comment']
